to do: 
    objectif métier 
    1) une classe( mère) abstraite record (load) → (class enfant) to_csv qui récup le df → pour l'enregistrer en CSV
    2) une classe ( mère) abstraite obj_client →(classe enfant)  ville, station , record  

Imperatif avant la prochaine seance:
    -EXTRACTION des données
    - Intégration des données dans différents Modèles ( ex: Station & Record)

Avant le 25/11/12

In [25]:
from extractors.meteo_toulouse_extractor import MeteoToulouseExtractor
from extractors.meteo_toulouse_extractor import MeteoToulouseExtractor
from validators.dataframe_validator import DataFrameValidator
import pandas as pd 
from abc import ABC, abstractmethod
from extractors.stations_config import STATIONS

In [ ]:
class transform(ABC):
    """ Classe abstraite pour avoir plusieurs possibilités de transform""" 
    
    @abstractmethod
    def transform(self):
        """Méthode de transform à implémenter"""
        pass 


In [ ]:
import os
from datetime import datetime
import pandas as pd

class CsvTransform:
    """Transformation/validation/sauvegarde d'un DataFrame en CSV."""

    def __init__(self, df: pd.DataFrame, station: str):
        self.df = df.copy()
        self.station = station

    def validate(self):
        #Appelle validateur
        DataFrameValidator.validate(self.df)
        print(f"Donnees valides pour {self.station} ({len(self.df)} lignes)")




In [ ]:
def load_data_df(station: str) -> pd.DataFrame:
    """Extrait et valide les données d'une station donnée."""
    extractor = MeteoToulouseExtractor(station)
    data_json = extractor.extract()
    df = extractor.to_dataframe(data_json)

    # Validation simple
    try:
        DataFrameValidator.validate(df)
        print(f"✅ Données valides pour {station} ({len(df)} lignes)")
    except Exception as e:
        print(f"⚠️  Erreur dans la validation de {station} :", e)

    return df


def merge_all_stations() -> pd.DataFrame:
    """Concatène les DataFrames de toutes les stations disponibles."""
    list_df = []

    for station in STATIONS.keys():
        try:
            print(f"🔍 Extraction en cours pour : {station}")
            df_station = load_data_df(station)

            # Ajout d'une colonne pour identifier la station dans la fusion
            df_station["station_name"] = station

            list_df.append(df_station)

        except Exception as e:
            print(f"❌ Erreur pour {station} → pas grave, on continue ({e})")

    # Fusionner toutes les stations sans erreur
    if list_df:
        df_final = pd.concat(list_df, ignore_index=True)
        print(f"\n🎯 Fusion terminée : {len(df_final)} lignes au total.")
        return df_final
    else:
        print("🚫 Aucune donnée n’a pu être extraite.")
        return pd.DataFrame()


In [23]:
df = merge_all_stations()

🔍 Extraction en cours pour : montaudran
✅ Données valides pour montaudran (96 lignes)
🔍 Extraction en cours pour : colomiers_zi_enjacca
✅ Données valides pour colomiers_zi_enjacca (95 lignes)
🔍 Extraction en cours pour : parc_maourine
✅ Données valides pour parc_maourine (88 lignes)
🔍 Extraction en cours pour : marengo
❌ Erreur pour marengo → pas grave, on continue (Erreur API 404: {
  "error_code": "NotFoundResource",
  "message": "The requested dataset 2-station-meteo-toulouse-marengo does not exist."
})
🔍 Extraction en cours pour : pech_david
✅ Données valides pour pech_david (78 lignes)
🔍 Extraction en cours pour : compans_cafarelli
✅ Données valides pour compans_cafarelli (96 lignes)
🔍 Extraction en cours pour : fondeyre
✅ Données valides pour fondeyre (14 lignes)
🔍 Extraction en cours pour : paul_sabatier
✅ Données valides pour paul_sabatier (92 lignes)
🔍 Extraction en cours pour : mondouzil
⚠️  Erreur dans la validation de mondouzil : Le DataFrame est vide
🔍 Extraction en cours 

Point d'amélioration, si la data éxiste dejas ne la rajoute pas au CSV
Log

In [24]:
df.to_csv("data_meteo_toulouse_station/data_csv.csv")

In [ ]:
if __name__ == "__main__":
    transform = CsvTransform()
    df_all = transform.transform()   
    transform.save_to_csv()          

🚀 Démarrage de la transformation (ALL stations)...
📡 Extraction en cours pour : montaudran
📡 Extraction en cours pour : colomiers_zi_enjacca
📡 Extraction en cours pour : parc_maourine
📡 Extraction en cours pour : marengo
⚠️ Erreur pour marengo → ignorée (Erreur API 404: {
  "error_code": "NotFoundResource",
  "message": "The requested dataset 2-station-meteo-toulouse-marengo does not exist."
})
📡 Extraction en cours pour : pech_david
📡 Extraction en cours pour : compans_cafarelli
📡 Extraction en cours pour : fondeyre
📡 Extraction en cours pour : paul_sabatier
📡 Extraction en cours pour : mondouzil
⚠️ Erreur pour mondouzil → ignorée (Le DataFrame est vide)
📡 Extraction en cours pour : basso_cambo
⚠️ Erreur pour basso_cambo → ignorée (Erreur API 404: {
  "error_code": "NotFoundResource",
  "message": "The requested dataset 8-station-meteo-toulouse-basso-cambo does not exist."
})
📡 Extraction en cours pour : soupetard
📡 Extraction en cours pour : mons_epuration
📡 Extraction en cours pour 